# Limpieza archivo CSV (Fuente: Tripadvisor):

Una vez tenemos el archivos csv,procedemos a limpiarlo.

In [7]:
import pandas as pd

import sys
sys.path.append("..")
from src.funciones import * 

In [8]:
ruta_al_csv = '../data/Tripadvisor2.csv'
df = pd.read_csv(ruta_al_csv)
data=df.copy()

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       281 non-null    object
 1   1       281 non-null    object
 2   2       281 non-null    object
 3   3       25 non-null     object
 4   4       2 non-null      object
dtypes: object(5)
memory usage: 11.1+ KB


In [10]:
data.describe().T

,count,unique,top,freq
0,281,204,MICHELIN,14
1,281,206,Viajes,11
2,281,95,"Italiana, Pizza€€ - €€€Carta",29
3,25,9,Iniciar sesión,9
4,2,1,Iniciar sesión,2


In [11]:
data.head()

,0,1,2,3,4
0,Opinión,Viajes,Alertas,Iniciar sesión,NaN
1,Opinión,Viajes,Alertas,Iniciar sesión,NaN
2,31. Pizzeria Fratelli,460 opinionesAbre en 6 minutos,"Italiana, Pizza€€ - €€€Carta",NaN,NaN
3,32. Asian Army,52 opiniones,"Asiática, Vietnamita€€ - €€€",NaN,NaN
4,33. Menomale,204 opinionesCerrado ahora,"Italiana, Pizza€€ - €€€Carta",NaN,NaN


Tras ver el dataframe procedemos con la siguiente limpieza:

* Ponemos los títulos bien
* Quitamos las filas que han salido mal durante la extracción de datos
* Quitamos los números que preceden a los nomber
* Reseteamos los índices
* Quitamos de la columna opiniones Abierto/Cerrado ahora
* Quitamos de Tipo '€' y los añadimos a precio la equivalencia
* Quitamos Carta de Tipo

In [12]:
data.columns=['Nombre','Opiniones','Tipo','Precio','Cosas']

eliminar=['Patrocinado', 'Descubrir','MICHELIN','Opinión']
data = data[~data['Nombre'].isin(eliminar)]

data['Nombre'] = data['Nombre'].str.replace(r'\d+\.\s', '', regex=True) #eliminamos numeros de nombre
data = data.reset_index(drop=True)

data['Opiniones'] = data['Opiniones'].str.replace('Abierto ahora', '')
data['Opiniones'] = data['Opiniones'].str.replace('Cerrado ahora', '')

data['Tipo'] = data['Tipo'].str.replace(r'Carta$', '', regex=True)
data['Precio'] = data['Tipo'].str.extract(r'(€+)')

In [13]:
def precio(tipo):
    if '€€€€' in tipo:
        return 'Alto'
    elif '€€ - €€€' in tipo:
        return 'Medio'
    elif '€' in tipo:
        return 'Bajo'
    else:
        return ''

In [14]:
data['Precio'] = data['Tipo'].apply(precio)
data['Tipo'] = data['Tipo'].str.replace('€', '', regex=False)
data['Tipo'] = data['Tipo'].str.replace('-', '', regex=False)

In [15]:
data.head()

,Nombre,Opiniones,Tipo,Precio,Cosas
0,Pizzeria Fratelli,460 opinionesAbre en 6 minutos,"Italiana, Pizza",Medio,NaN
1,Asian Army,52 opiniones,"Asiática, Vietnamita",Medio,NaN
2,Menomale,204 opiniones,"Italiana, Pizza",Medio,NaN
3,Borax,445 opinionesCerrado hoy,"Caribeña, Latina",Medio,NaN
4,Éccolo,133 opiniones,"Italiana, Mediterránea",Medio,NaN


Ponemos valores únicos en Tipo:

In [16]:
limpieza_tipo(data)

In [17]:
data.Tipo.value_counts()

Tipo
Italiana                        108
Japonesa                         47
China                            20
India                            15
China                            10
Japonesa                          9
Mexicana                          6
Mexicana                          6
Caribeña                          5
Peruana                           4
Vietnamita                        3
Fusión                            2
Japonesa rápida                   2
Tailandesa                        2
Caribeña                          2
Mediterránea                      2
Venezolana                        1
Americana rápida                  1
Colombiana                        1
Venezolana                        1
Filipina                          1
LatinaComida rápida               1
LatinaInternacional               1
Comida rápidaInternacional        1
FusiónSaludable                   1
Argentino                         1
Americana                         1
Coreana                

In [18]:
data.tail()

,Nombre,Opiniones,Tipo,Precio,Cosas
251,Ronda 14,982 opiniones,Japonesa,Medio,NaN
252,PICCOLO DIAVOLO,688 opiniones,Italiana,Medio,NaN
253,La Piccola Trattoria,331 opiniones,Italiana,Medio,NaN
254,Kzen,208 opiniones,China,Medio,NaN
255,Il Boccone,179 opinionesCerrado hoy,Italiana,Medio,NaN


In [19]:
optim(data)

,Nombre,Opiniones,Tipo,Precio,Cosas
0,Pizzeria Fratelli,460 opinionesAbre en 6 minutos,Italiana,Medio,NaN
1,Asian Army,52 opiniones,Vietnamita,Medio,NaN
2,Menomale,204 opiniones,Italiana,Medio,NaN
3,Borax,445 opinionesCerrado hoy,Caribeña,Medio,NaN
4,Éccolo,133 opiniones,Italiana,Medio,NaN
...,...,...,...,...,...
251,Ronda 14,982 opiniones,Japonesa,Medio,NaN
252,PICCOLO DIAVOLO,688 opiniones,Italiana,Medio,NaN
253,La Piccola Trattoria,331 opiniones,Italiana,Medio,NaN
254,Kzen,208 opiniones,China,Medio,NaN


In [21]:
data.to_json('../data/restautante.json')


### Veo que han quedado cosas raras en la columna rótulo, pero no preocuparse, luego cogere el resto de las tablas y solo me quedare con las que coincidan y el resto fuera ✂️✂️x